# HijackSpam
Looking for the use of a domain anywhere.

[Per-wiki results](#Results) - [Summary table](#Summary)

In [1]:
import time

import requests
from IPython import display
import pywikibot
from pywikibot import pagegenerators

def get_sitematrix():
    """Request the sitematrix from the API, check if open, then yeild URLs"""
    
    def check_status(checksite):
        """Return true only if wiki is public and open"""
        return ((checksite.get('closed') is None)
            and (checksite.get('private') is None)
            and (checksite.get('fishbowl') is None))
    
    
    payload = {"action": "sitematrix", "format": "json",
               "smlangprop": "site", "smsiteprop": "url",}
    headers = {'user-agent':
               'HijackSpam on PAWS; User:AntiCompositeBot, pywikibot/'
               + pywikibot.__version__}
    url = 'https://meta.wikimedia.org/w/api.php'
    r = requests.get(url, headers=headers, params=payload)
    r.raise_for_status()
    
    result = r.json()['sitematrix']
    for key, lang in result.items():
        if key == 'count':
            continue
        elif key == 'specials':
            for site in lang:
                if check_status(site):
                    yield site['url']
        else:
            for site in lang['site']:
                if check_status(site):
                    yield site['url']


def list_pages(site, target):
    """Takes a pywikibot site object and yeilds the pages linking to the target"""
    
    for num in range(0,4):
        if num % 2 == 0:
            protocol = 'http'
        else:
            protocol = 'https'
            
        if num > 1:
            ctar = '*.' + target
        else:
            ctar = target
        
        for page in pywikibot.pagegenerators.LinksearchPageGenerator(
            ctar, site=site, protocol=protocol):
            yield page

            
def output(text):
    display.display_markdown(text, raw=True)
    
    
def prep_markdown(pages, site):
    md = '' 
    count = 0
    for page in pages:
        count += 1
        md += f'* [{page.title()}]({page.full_url()})\n'
    if count > 0:
        md = f'## {site.dbName()}: {count} \n' + md
        output(md)
        return count

    
def summary_table(counts):
    tot = 0
    md = '## Summary\n\n|Wiki|Count|\n|---|---|\n'
    wt = '{| class="wikitable"\n|-\n! Wiki !! Count !! Volunteer !! Progress\n'
    for wiki, count in counts.items():
        if count is not None:
            md += f'|{wiki}|{str(count)}|\n'
            wt += f'|-\n| {wiki} || {str(count)} || || \n'
            tot += count
    
    wt += '|}'
    md += '\nTotal: ' + str(tot) + '\n```\n' + wt + '\n```\n'
    output(md)
    
        
def main():
    target = 'blackwell-synergy.com'
        
    counts = {}
    try:
        sitematrix = get_sitematrix()
    except requests.exceptions:
        time.sleep(5)
        sitematrix = get_sitematrix()
        
    output('Scanning all public wikis for ' + target + ' at ' + time.asctime() + '\n')
    for url in sitematrix:
        try:
            cur_site = pywikibot.Site(url=url + '/wiki/MediaWiki:Delete/en')
        except Exception:
            output('Skipping ' + url)
            continue
            
        pages = list_pages(cur_site, target)
        counts[cur_site.dbName()] = prep_markdown(pages, cur_site)
        
    summary_table(counts)
    output('Finished')

# Results

In [2]:
main()

Scanning all public wikis for blackwell-synergy.com at Thu Oct 24 17:01:03 2019


## alswiki: 5 
* [Diskussion:Mesomycetozoa](https://als.wikipedia.org/wiki/Diskussion%3AMesomycetozoa)
* [Diskussion:Kragengeißeltierchen](https://als.wikipedia.org/wiki/Diskussion%3AKragengei%C3%9Feltierchen)
* [Diskussion:Chloroplastida](https://als.wikipedia.org/wiki/Diskussion%3AChloroplastida)
* [Diskussion:Streptophytina](https://als.wikipedia.org/wiki/Diskussion%3AStreptophytina)
* [Diskussion:Chlorophyta](https://als.wikipedia.org/wiki/Diskussion%3AChlorophyta)


## arwiki: 131 
* [مستخدم:Emijrp/External Links Ranking](https://ar.wikipedia.org/wiki/%D9%85%D8%B3%D8%AA%D8%AE%D8%AF%D9%85%3AEmijrp%2FExternal_Links_Ranking)
* [مستخدم:Emijrp/External Links Ranking](https://ar.wikipedia.org/wiki/%D9%85%D8%B3%D8%AA%D8%AE%D8%AF%D9%85%3AEmijrp%2FExternal_Links_Ranking)
* [زهري (مرض)](https://ar.wikipedia.org/wiki/%D8%B2%D9%87%D8%B1%D9%8A_%28%D9%85%D8%B1%D8%B6%29)
* [أيام العرب](https://ar.wikipedia.org/wiki/%D8%A3%D9%8A%D8%A7%D9%85_%D8%A7%D9%84%D8%B9%D8%B1%D8%A8)
* [بكتيريا قديمة](https://ar.wikipedia.org/wiki/%D8%A8%D9%83%D8%AA%D9%8A%D8%B1%D9%8A%D8%A7_%D9%82%D8%AF%D9%8A%D9%85%D8%A9)
* [السكري](https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%B3%D9%83%D8%B1%D9%8A)
* [عجز صوتي](https://ar.wikipedia.org/wiki/%D8%B9%D8%AC%D8%B2_%D8%B5%D9%88%D8%AA%D9%8A)
* [حاصل الذكاء](https://ar.wikipedia.org/wiki/%D8%AD%D8%A7%D8%B5%D9%84_%D8%A7%D9%84%D8%B0%D9%83%D8%A7%D8%A1)
* [فرس النهر](https://ar.wikipedia.org/wiki/%D9%81%D8%B1%D8%B3_%D8%A7%D9%84%D9%86%D9%87%D8%B1)
* [دورة حياة معقدات القمة](https://ar.wikipedia.org/wiki/%D8%AF%D9%88%D8%B1%D8%A9_%D8%AD%D9%8A%D8%A7%D8%A9_%D9%85%D8%B9%D9%82%D8%AF%D8%A7%D8%AA_%D8%A7%D9%84%D9%82%D9%85%D8%A9)
* [تقاسم النماء الجزئي](https://ar.wikipedia.org/wiki/%D8%AA%D9%82%D8%A7%D8%B3%D9%85_%D8%A7%D9%84%D9%86%D9%85%D8%A7%D8%A1_%D8%A7%D9%84%D8%AC%D8%B2%D8%A6%D9%8A)
* [اضطراب نفسي](https://ar.wikipedia.org/wiki/%D8%A7%D8%B6%D8%B7%D8%B1%D8%A7%D8%A8_%D9%86%D9%81%D8%B3%D9%8A)
* [اسوداد الجلد](https://ar.wikipedia.org/wiki/%D8%A7%D8%B3%D9%88%D8%AF%D8%A7%D8%AF_%D8%A7%D9%84%D8%AC%D9%84%D8%AF)
* [افتراس](https://ar.wikipedia.org/wiki/%D8%A7%D9%81%D8%AA%D8%B1%D8%A7%D8%B3)
* [معركة سندويتش (1460)](https://ar.wikipedia.org/wiki/%D9%85%D8%B9%D8%B1%D9%83%D8%A9_%D8%B3%D9%86%D8%AF%D9%88%D9%8A%D8%AA%D8%B4_%281460%29)
* [مخلوقات النبض](https://ar.wikipedia.org/wiki/%D9%85%D8%AE%D9%84%D9%88%D9%82%D8%A7%D8%AA_%D8%A7%D9%84%D9%86%D8%A8%D8%B6)
* [اليهود المزراحيون في إسرائيل](https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D9%8A%D9%87%D9%88%D8%AF_%D8%A7%D9%84%D9%85%D8%B2%D8%B1%D8%A7%D8%AD%D9%8A%D9%88%D9%86_%D9%81%D9%8A_%D8%A5%D8%B3%D8%B1%D8%A7%D8%A6%D9%8A%D9%84)
* [إسرائيليون](https://ar.wikipedia.org/wiki/%D8%A5%D8%B3%D8%B1%D8%A7%D8%A6%D9%8A%D9%84%D9%8A%D9%88%D9%86)
* [رستيونية](https://ar.wikipedia.org/wiki/%D8%B1%D8%B3%D8%AA%D9%8A%D9%88%D9%86%D9%8A%D8%A9)
* [تزلج على الجليد](https://ar.wikipedia.org/wiki/%D8%AA%D8%B2%D9%84%D8%AC_%D8%B9%D9%84%D9%89_%D8%A7%D9%84%D8%AC%D9%84%D9%8A%D8%AF)
* [حمري](https://ar.wikipedia.org/wiki/%D8%AD%D9%85%D8%B1%D9%8A)
* [ملدن](https://ar.wikipedia.org/wiki/%D9%85%D9%84%D8%AF%D9%86)
* [متحف بيروت الوطني](https://ar.wikipedia.org/wiki/%D9%85%D8%AA%D8%AD%D9%81_%D8%A8%D9%8A%D8%B1%D9%88%D8%AA_%D8%A7%D9%84%D9%88%D8%B7%D9%86%D9%8A)
* [نمر](https://ar.wikipedia.org/wiki/%D9%86%D9%85%D8%B1)
* [جرعة فيتامين ج هائلة](https://ar.wikipedia.org/wiki/%D8%AC%D8%B1%D8%B9%D8%A9_%D9%81%D9%8A%D8%AA%D8%A7%D9%85%D9%8A%D9%86_%D8%AC_%D9%87%D8%A7%D8%A6%D9%84%D8%A9)
* [مقاومة المضادات الحيوية](https://ar.wikipedia.org/wiki/%D9%85%D9%82%D8%A7%D9%88%D9%85%D8%A9_%D8%A7%D9%84%D9%85%D8%B6%D8%A7%D8%AF%D8%A7%D8%AA_%D8%A7%D9%84%D8%AD%D9%8A%D9%88%D9%8A%D8%A9)
* [تقويم هولوسين](https://ar.wikipedia.org/wiki/%D8%AA%D9%82%D9%88%D9%8A%D9%85_%D9%87%D9%88%D9%84%D9%88%D8%B3%D9%8A%D9%86)
* [بحيرة كاريبا](https://ar.wikipedia.org/wiki/%D8%A8%D8%AD%D9%8A%D8%B1%D8%A9_%D9%83%D8%A7%D8%B1%D9%8A%D8%A8%D8%A7)
* [فاصولياء](https://ar.wikipedia.org/wiki/%D9%81%D8%A7%D8%B5%D9%88%D9%84%D9%8A%D8%A7%D8%A1)
* [ثيوبرومين](https://ar.wikipedia.org/wiki/%D8%AB%D9%8A%D9%88%D8%A8%D8%B1%D9%88%D9%85%D9%8A%D9%86)
* [قائمة مجرات المجموعة المحلية](https://ar.wikipedia.org/wiki/%D9%82%D8%A7%D8%A6%D9%85%D8%A9_%D9%85%D8%AC%D8%B1%D8%A7%D8%AA_%D8%A7%D9%84%D9%85%D8%AC%D9%85%D9%88%D8%B9%D8%A9_%D8%A7%D9%84%D9%85%D8%AD%D9%84%D9%8A%D8%A9)
* [أدنى كتلة](https://ar.wikipedia.org/wiki/%D8%A3%D8%AF%D9%86%D9%89_%D9%83%D8%AA%D9%84%D8%A9)
* [ليليات](https://ar.wikipedia.org/wiki/%D9%84%D9%8A%D9%84%D9%8A%D8%A7%D8%AA)
* [أغذية معدلة وراثيا](https://ar.wikipedia.org/wiki/%D8%A3%D8%BA%D8%B0%D9%8A%D8%A9_%D9%85%D8%B9%D8%AF%D9%84%D8%A9_%D9%88%D8%B1%D8%A7%D8%AB%D9%8A%D8%A7)
* [ثعلبة ذكرية الشكل](https://ar.wikipedia.org/wiki/%D8%AB%D8%B9%D9%84%D8%A8%D8%A9_%D8%B0%D9%83%D8%B1%D9%8A%D8%A9_%D8%A7%D9%84%D8%B4%D9%83%D9%84)
* [ذئب إيراني](https://ar.wikipedia.org/wiki/%D8%B0%D8%A6%D8%A8_%D8%A5%D9%8A%D8%B1%D8%A7%D9%86%D9%8A)
* [ذئب رمادي](https://ar.wikipedia.org/wiki/%D8%B0%D8%A6%D8%A8_%D8%B1%D9%85%D8%A7%D8%AF%D9%8A)
* [كلب (جنس)](https://ar.wikipedia.org/wiki/%D9%83%D9%84%D8%A8_%28%D8%AC%D9%86%D8%B3%29)
* [فرضية تطوير القدرة التنافسية المتزايدة](https://ar.wikipedia.org/wiki/%D9%81%D8%B1%D8%B6%D9%8A%D8%A9_%D8%AA%D8%B7%D9%88%D9%8A%D8%B1_%D8%A7%D9%84%D9%82%D8%AF%D8%B1%D8%A9_%D8%A7%D9%84%D8%AA%D9%86%D8%A7%D9%81%D8%B3%D9%8A%D8%A9_%D8%A7%D9%84%D9%85%D8%AA%D8%B2%D8%A7%D9%8A%D8%AF%D8%A9)
* [نشوء الفيروسات](https://ar.wikipedia.org/wiki/%D9%86%D8%B4%D9%88%D8%A1_%D8%A7%D9%84%D9%81%D9%8A%D8%B1%D9%88%D8%B3%D8%A7%D8%AA)
* [فيروس](https://ar.wikipedia.org/wiki/%D9%81%D9%8A%D8%B1%D9%88%D8%B3)
* [جيرمي بنثام](https://ar.wikipedia.org/wiki/%D8%AC%D9%8A%D8%B1%D9%85%D9%8A_%D8%A8%D9%86%D8%AB%D8%A7%D9%85)
* [حقوق الحيوان](https://ar.wikipedia.org/wiki/%D8%AD%D9%82%D9%88%D9%82_%D8%A7%D9%84%D8%AD%D9%8A%D9%88%D8%A7%D9%86)
* [فرضية استبعاد التشويش الحسي](https://ar.wikipedia.org/wiki/%D9%81%D8%B1%D8%B6%D9%8A%D8%A9_%D8%A7%D8%B3%D8%AA%D8%A8%D8%B9%D8%A7%D8%AF_%D8%A7%D9%84%D8%AA%D8%B4%D9%88%D9%8A%D8%B4_%D8%A7%D9%84%D8%AD%D8%B3%D9%8A)
* [أنثروبولوجيا ثقافية حيوية](https://ar.wikipedia.org/wiki/%D8%A3%D9%86%D8%AB%D8%B1%D9%88%D8%A8%D9%88%D9%84%D9%88%D8%AC%D9%8A%D8%A7_%D8%AB%D9%82%D8%A7%D9%81%D9%8A%D8%A9_%D8%AD%D9%8A%D9%88%D9%8A%D8%A9)
* [شادن قزم](https://ar.wikipedia.org/wiki/%D8%B4%D8%A7%D8%AF%D9%86_%D9%82%D8%B2%D9%85)
* [ستروثيوميمس](https://ar.wikipedia.org/wiki/%D8%B3%D8%AA%D8%B1%D9%88%D8%AB%D9%8A%D9%88%D9%85%D9%8A%D9%85%D8%B3)
* [أندريه هوفمان (سياسي)](https://ar.wikipedia.org/wiki/%D8%A3%D9%86%D8%AF%D8%B1%D9%8A%D9%87_%D9%87%D9%88%D9%81%D9%85%D8%A7%D9%86_%28%D8%B3%D9%8A%D8%A7%D8%B3%D9%8A%29)
* [حزب العمل الاشتراكي العربي](https://ar.wikipedia.org/wiki/%D8%AD%D8%B2%D8%A8_%D8%A7%D9%84%D8%B9%D9%85%D9%84_%D8%A7%D9%84%D8%A7%D8%B4%D8%AA%D8%B1%D8%A7%D9%83%D9%8A_%D8%A7%D9%84%D8%B9%D8%B1%D8%A8%D9%8A)
* [طائر](https://ar.wikipedia.org/wiki/%D8%B7%D8%A7%D8%A6%D8%B1)
* [غابابنتين](https://ar.wikipedia.org/wiki/%D8%BA%D8%A7%D8%A8%D8%A7%D8%A8%D9%86%D8%AA%D9%8A%D9%86)
* [كمثريات](https://ar.wikipedia.org/wiki/%D9%83%D9%85%D8%AB%D8%B1%D9%8A%D8%A7%D8%AA)
* [تنغستات الكادميوم](https://ar.wikipedia.org/wiki/%D8%AA%D9%86%D8%BA%D8%B3%D8%AA%D8%A7%D8%AA_%D8%A7%D9%84%D9%83%D8%A7%D8%AF%D9%85%D9%8A%D9%88%D9%85)
* [ثنائي كرومات البوتاسيوم](https://ar.wikipedia.org/wiki/%D8%AB%D9%86%D8%A7%D8%A6%D9%8A_%D9%83%D8%B1%D9%88%D9%85%D8%A7%D8%AA_%D8%A7%D9%84%D8%A8%D9%88%D8%AA%D8%A7%D8%B3%D9%8A%D9%88%D9%85)
* [ريكي (علاج الطاقة)](https://ar.wikipedia.org/wiki/%D8%B1%D9%8A%D9%83%D9%8A_%28%D8%B9%D9%84%D8%A7%D8%AC_%D8%A7%D9%84%D8%B7%D8%A7%D9%82%D8%A9%29)
* [تحفيز الحلمة](https://ar.wikipedia.org/wiki/%D8%AA%D8%AD%D9%81%D9%8A%D8%B2_%D8%A7%D9%84%D8%AD%D9%84%D9%85%D8%A9)
* [حلمة](https://ar.wikipedia.org/wiki/%D8%AD%D9%84%D9%85%D8%A9)
* [جراحة القلب والصدر](https://ar.wikipedia.org/wiki/%D8%AC%D8%B1%D8%A7%D8%AD%D8%A9_%D8%A7%D9%84%D9%82%D9%84%D8%A8_%D9%88%D8%A7%D9%84%D8%B5%D8%AF%D8%B1)
* [هيلين كالديكوت](https://ar.wikipedia.org/wiki/%D9%87%D9%8A%D9%84%D9%8A%D9%86_%D9%83%D8%A7%D9%84%D8%AF%D9%8A%D9%83%D9%88%D8%AA)
* [عوز فيتامين سي](https://ar.wikipedia.org/wiki/%D8%B9%D9%88%D8%B2_%D9%81%D9%8A%D8%AA%D8%A7%D9%85%D9%8A%D9%86_%D8%B3%D9%8A)
* [قائمة النجوم الزائفة](https://ar.wikipedia.org/wiki/%D9%82%D8%A7%D8%A6%D9%85%D8%A9_%D8%A7%D9%84%D9%86%D8%AC%D9%88%D9%85_%D8%A7%D9%84%D8%B2%D8%A7%D8%A6%D9%81%D8%A9)
* [شخير](https://ar.wikipedia.org/wiki/%D8%B4%D8%AE%D9%8A%D8%B1)
* [انسلاخيات](https://ar.wikipedia.org/wiki/%D8%A7%D9%86%D8%B3%D9%84%D8%A7%D8%AE%D9%8A%D8%A7%D8%AA)
* [إصابة دماغية رضية](https://ar.wikipedia.org/wiki/%D8%A5%D8%B5%D8%A7%D8%A8%D8%A9_%D8%AF%D9%85%D8%A7%D8%BA%D9%8A%D8%A9_%D8%B1%D8%B6%D9%8A%D8%A9)
* [بريميدون](https://ar.wikipedia.org/wiki/%D8%A8%D8%B1%D9%8A%D9%85%D9%8A%D8%AF%D9%88%D9%86)
* [نوبة (طب)](https://ar.wikipedia.org/wiki/%D9%86%D9%88%D8%A8%D8%A9_%28%D8%B7%D8%A8%29)
* [زونيساميد](https://ar.wikipedia.org/wiki/%D8%B2%D9%88%D9%86%D9%8A%D8%B3%D8%A7%D9%85%D9%8A%D8%AF)
* [سرطان المريء](https://ar.wikipedia.org/wiki/%D8%B3%D8%B1%D8%B7%D8%A7%D9%86_%D8%A7%D9%84%D9%85%D8%B1%D9%8A%D8%A1)
* [علم الأوبئة المتعلق بمرض الإيدز](https://ar.wikipedia.org/wiki/%D8%B9%D9%84%D9%85_%D8%A7%D9%84%D8%A3%D9%88%D8%A8%D8%A6%D8%A9_%D8%A7%D9%84%D9%85%D8%AA%D8%B9%D9%84%D9%82_%D8%A8%D9%85%D8%B1%D8%B6_%D8%A7%D9%84%D8%A5%D9%8A%D8%AF%D8%B2)
* [نقاش:فلاديمير لينين/أرشيف 1](https://ar.wikipedia.org/wiki/%D9%86%D9%82%D8%A7%D8%B4%3A%D9%81%D9%84%D8%A7%D8%AF%D9%8A%D9%85%D9%8A%D8%B1_%D9%84%D9%8A%D9%86%D9%8A%D9%86%2F%D8%A3%D8%B1%D8%B4%D9%8A%D9%81_1)
* [قوس بلعومية](https://ar.wikipedia.org/wiki/%D9%82%D9%88%D8%B3_%D8%A8%D9%84%D8%B9%D9%88%D9%85%D9%8A%D8%A9)
* [فرط ضغط الدم الرئوي](https://ar.wikipedia.org/wiki/%D9%81%D8%B1%D8%B7_%D8%B6%D8%BA%D8%B7_%D8%A7%D9%84%D8%AF%D9%85_%D8%A7%D9%84%D8%B1%D8%A6%D9%88%D9%8A)
* [اختصاص فرعي](https://ar.wikipedia.org/wiki/%D8%A7%D8%AE%D8%AA%D8%B5%D8%A7%D8%B5_%D9%81%D8%B1%D8%B9%D9%8A)
* [زيدوفودين](https://ar.wikipedia.org/wiki/%D8%B2%D9%8A%D8%AF%D9%88%D9%81%D9%88%D8%AF%D9%8A%D9%86)
* [علم جينوم الصحة العامة](https://ar.wikipedia.org/wiki/%D8%B9%D9%84%D9%85_%D8%AC%D9%8A%D9%86%D9%88%D9%85_%D8%A7%D9%84%D8%B5%D8%AD%D8%A9_%D8%A7%D9%84%D8%B9%D8%A7%D9%85%D8%A9)
* [نمر (جنس)](https://ar.wikipedia.org/wiki/%D9%86%D9%85%D8%B1_%28%D8%AC%D9%86%D8%B3%29)
* [اسفير](https://ar.wikipedia.org/wiki/%D8%A7%D8%B3%D9%81%D9%8A%D8%B1)
* [كراسين](https://ar.wikipedia.org/wiki/%D9%83%D8%B1%D8%A7%D8%B3%D9%8A%D9%86)
* [أوستاريفيات](https://ar.wikipedia.org/wiki/%D8%A3%D9%88%D8%B3%D8%AA%D8%A7%D8%B1%D9%8A%D9%81%D9%8A%D8%A7%D8%AA)
* [اللوبي الإسرائيلي والسياسة الخارجية الأمريكية (كتاب)](https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D9%84%D9%88%D8%A8%D9%8A_%D8%A7%D9%84%D8%A5%D8%B3%D8%B1%D8%A7%D8%A6%D9%8A%D9%84%D9%8A_%D9%88%D8%A7%D9%84%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9_%D8%A7%D9%84%D8%AE%D8%A7%D8%B1%D8%AC%D9%8A%D8%A9_%D8%A7%D9%84%D8%A3%D9%85%D8%B1%D9%8A%D9%83%D9%8A%D8%A9_%28%D9%83%D8%AA%D8%A7%D8%A8%29)
* [جيوفاني فابر](https://ar.wikipedia.org/wiki/%D8%AC%D9%8A%D9%88%D9%81%D8%A7%D9%86%D9%8A_%D9%81%D8%A7%D8%A8%D8%B1)
* [فيغاباترين](https://ar.wikipedia.org/wiki/%D9%81%D9%8A%D8%BA%D8%A7%D8%A8%D8%A7%D8%AA%D8%B1%D9%8A%D9%86)
* [مسألة العموميات](https://ar.wikipedia.org/wiki/%D9%85%D8%B3%D8%A3%D9%84%D8%A9_%D8%A7%D9%84%D8%B9%D9%85%D9%88%D9%85%D9%8A%D8%A7%D8%AA)
* [تاريخ العالم](https://ar.wikipedia.org/wiki/%D8%AA%D8%A7%D8%B1%D9%8A%D8%AE_%D8%A7%D9%84%D8%B9%D8%A7%D9%84%D9%85)
* [أركيامفورا لونجيسيرفيا](https://ar.wikipedia.org/wiki/%D8%A3%D8%B1%D9%83%D9%8A%D8%A7%D9%85%D9%81%D9%88%D8%B1%D8%A7_%D9%84%D9%88%D9%86%D8%AC%D9%8A%D8%B3%D9%8A%D8%B1%D9%81%D9%8A%D8%A7)
* [لورازيبام](https://ar.wikipedia.org/wiki/%D9%84%D9%88%D8%B1%D8%A7%D8%B2%D9%8A%D8%A8%D8%A7%D9%85)
* [انفصال القزحية](https://ar.wikipedia.org/wiki/%D8%A7%D9%86%D9%81%D8%B5%D8%A7%D9%84_%D8%A7%D9%84%D9%82%D8%B2%D8%AD%D9%8A%D8%A9)
* [نمر فارسي](https://ar.wikipedia.org/wiki/%D9%86%D9%85%D8%B1_%D9%81%D8%A7%D8%B1%D8%B3%D9%8A)
* [ماغنولانيات](https://ar.wikipedia.org/wiki/%D9%85%D8%A7%D8%BA%D9%86%D9%88%D9%84%D8%A7%D9%86%D9%8A%D8%A7%D8%AA)
* [حبارى](https://ar.wikipedia.org/wiki/%D8%AD%D8%A8%D8%A7%D8%B1%D9%89)
* [قيمة عامة](https://ar.wikipedia.org/wiki/%D9%82%D9%8A%D9%85%D8%A9_%D8%B9%D8%A7%D9%85%D8%A9)
* [معالجة لب الأسنان](https://ar.wikipedia.org/wiki/%D9%85%D8%B9%D8%A7%D9%84%D8%AC%D8%A9_%D9%84%D8%A8_%D8%A7%D9%84%D8%A3%D8%B3%D9%86%D8%A7%D9%86)
* [حزاز مسطح](https://ar.wikipedia.org/wiki/%D8%AD%D8%B2%D8%A7%D8%B2_%D9%85%D8%B3%D8%B7%D8%AD)
* [ريتوكسيماب](https://ar.wikipedia.org/wiki/%D8%B1%D9%8A%D8%AA%D9%88%D9%83%D8%B3%D9%8A%D9%85%D8%A7%D8%A8)
* [سجل تحليل فعالية التكلفة](https://ar.wikipedia.org/wiki/%D8%B3%D8%AC%D9%84_%D8%AA%D8%AD%D9%84%D9%8A%D9%84_%D9%81%D8%B9%D8%A7%D9%84%D9%8A%D8%A9_%D8%A7%D9%84%D8%AA%D9%83%D9%84%D9%81%D8%A9)
* [متلازمة تولوزا - هنت](https://ar.wikipedia.org/wiki/%D9%85%D8%AA%D9%84%D8%A7%D8%B2%D9%85%D8%A9_%D8%AA%D9%88%D9%84%D9%88%D8%B2%D8%A7_-_%D9%87%D9%86%D8%AA)
* [عوز المناعة المشترك الشديد](https://ar.wikipedia.org/wiki/%D8%B9%D9%88%D8%B2_%D8%A7%D9%84%D9%85%D9%86%D8%A7%D8%B9%D8%A9_%D8%A7%D9%84%D9%85%D8%B4%D8%AA%D8%B1%D9%83_%D8%A7%D9%84%D8%B4%D8%AF%D9%8A%D8%AF)
* [هرمون وطائي نخامي](https://ar.wikipedia.org/wiki/%D9%87%D8%B1%D9%85%D9%88%D9%86_%D9%88%D8%B7%D8%A7%D8%A6%D9%8A_%D9%86%D8%AE%D8%A7%D9%85%D9%8A)
* [التهاب الدماغ والنخاع الحاد المنتشر](https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AA%D9%87%D8%A7%D8%A8_%D8%A7%D9%84%D8%AF%D9%85%D8%A7%D8%BA_%D9%88%D8%A7%D9%84%D9%86%D8%AE%D8%A7%D8%B9_%D8%A7%D9%84%D8%AD%D8%A7%D8%AF_%D8%A7%D9%84%D9%85%D9%86%D8%AA%D8%B4%D8%B1)
* [موروموناب - سي دي 3](https://ar.wikipedia.org/wiki/%D9%85%D9%88%D8%B1%D9%88%D9%85%D9%88%D9%86%D8%A7%D8%A8_-_%D8%B3%D9%8A_%D8%AF%D9%8A_3)
* [مستخدم:عمرو/التبرع بالدم 2](https://ar.wikipedia.org/wiki/%D9%85%D8%B3%D8%AA%D8%AE%D8%AF%D9%85%3A%D8%B9%D9%85%D8%B1%D9%88%2F%D8%A7%D9%84%D8%AA%D8%A8%D8%B1%D8%B9_%D8%A8%D8%A7%D9%84%D8%AF%D9%85_2)
* [سرطان الغدة الدرقية](https://ar.wikipedia.org/wiki/%D8%B3%D8%B1%D8%B7%D8%A7%D9%86_%D8%A7%D9%84%D8%BA%D8%AF%D8%A9_%D8%A7%D9%84%D8%AF%D8%B1%D9%82%D9%8A%D8%A9)
* [ريكتسيلات](https://ar.wikipedia.org/wiki/%D8%B1%D9%8A%D9%83%D8%AA%D8%B3%D9%8A%D9%84%D8%A7%D8%AA)
* [مثبط المدخل](https://ar.wikipedia.org/wiki/%D9%85%D8%AB%D8%A8%D8%B7_%D8%A7%D9%84%D9%85%D8%AF%D8%AE%D9%84)
* [مرض مناعي ذاتي](https://ar.wikipedia.org/wiki/%D9%85%D8%B1%D8%B6_%D9%85%D9%86%D8%A7%D8%B9%D9%8A_%D8%B0%D8%A7%D8%AA%D9%8A)
* [المعالجة بالتصليب](https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D9%85%D8%B9%D8%A7%D9%84%D8%AC%D8%A9_%D8%A8%D8%A7%D9%84%D8%AA%D8%B5%D9%84%D9%8A%D8%A8)
* [ناظمة قلبية اصطناعية](https://ar.wikipedia.org/wiki/%D9%86%D8%A7%D8%B8%D9%85%D8%A9_%D9%82%D9%84%D8%A8%D9%8A%D8%A9_%D8%A7%D8%B5%D8%B7%D9%86%D8%A7%D8%B9%D9%8A%D8%A9)
* [سن اليأس](https://ar.wikipedia.org/wiki/%D8%B3%D9%86_%D8%A7%D9%84%D9%8A%D8%A3%D8%B3)
* [إندوميتاسين](https://ar.wikipedia.org/wiki/%D8%A5%D9%86%D8%AF%D9%88%D9%85%D9%8A%D8%AA%D8%A7%D8%B3%D9%8A%D9%86)
* [رفرفة بطينية](https://ar.wikipedia.org/wiki/%D8%B1%D9%81%D8%B1%D9%81%D8%A9_%D8%A8%D8%B7%D9%8A%D9%86%D9%8A%D8%A9)
* [لمفوما بائية الخلايا للمنطقة الهامشية العقدية](https://ar.wikipedia.org/wiki/%D9%84%D9%85%D9%81%D9%88%D9%85%D8%A7_%D8%A8%D8%A7%D8%A6%D9%8A%D8%A9_%D8%A7%D9%84%D8%AE%D9%84%D8%A7%D9%8A%D8%A7_%D9%84%D9%84%D9%85%D9%86%D8%B7%D9%82%D8%A9_%D8%A7%D9%84%D9%87%D8%A7%D9%85%D8%B4%D9%8A%D8%A9_%D8%A7%D9%84%D8%B9%D9%82%D8%AF%D9%8A%D8%A9)
* [تشخيص بالاستبعاد](https://ar.wikipedia.org/wiki/%D8%AA%D8%B4%D8%AE%D9%8A%D8%B5_%D8%A8%D8%A7%D9%84%D8%A7%D8%B3%D8%AA%D8%A8%D8%B9%D8%A7%D8%AF)
* [الصرع الرولاندي](https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%B5%D8%B1%D8%B9_%D8%A7%D9%84%D8%B1%D9%88%D9%84%D8%A7%D9%86%D8%AF%D9%8A)
* [بساط شفاف](https://ar.wikipedia.org/wiki/%D8%A8%D8%B3%D8%A7%D8%B7_%D8%B4%D9%81%D8%A7%D9%81)
* [شرى البرد](https://ar.wikipedia.org/wiki/%D8%B4%D8%B1%D9%89_%D8%A7%D9%84%D8%A8%D8%B1%D8%AF)
* [سكوراد](https://ar.wikipedia.org/wiki/%D8%B3%D9%83%D9%88%D8%B1%D8%A7%D8%AF)
* [متلازمة سوتو](https://ar.wikipedia.org/wiki/%D9%85%D8%AA%D9%84%D8%A7%D8%B2%D9%85%D8%A9_%D8%B3%D9%88%D8%AA%D9%88)
* [نقص حمض يوريك الدم](https://ar.wikipedia.org/wiki/%D9%86%D9%82%D8%B5_%D8%AD%D9%85%D8%B6_%D9%8A%D9%88%D8%B1%D9%8A%D9%83_%D8%A7%D9%84%D8%AF%D9%85)
* [متلازمة السيروتونين](https://ar.wikipedia.org/wiki/%D9%85%D8%AA%D9%84%D8%A7%D8%B2%D9%85%D8%A9_%D8%A7%D9%84%D8%B3%D9%8A%D8%B1%D9%88%D8%AA%D9%88%D9%86%D9%8A%D9%86)
* [التهاب الكبد G](https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AA%D9%87%D8%A7%D8%A8_%D8%A7%D9%84%D9%83%D8%A8%D8%AF_G)
* [متلازمة بلومر فنسون](https://ar.wikipedia.org/wiki/%D9%85%D8%AA%D9%84%D8%A7%D8%B2%D9%85%D8%A9_%D8%A8%D9%84%D9%88%D9%85%D8%B1_%D9%81%D9%86%D8%B3%D9%88%D9%86)
* [نيسرية](https://ar.wikipedia.org/wiki/%D9%86%D9%8A%D8%B3%D8%B1%D9%8A%D8%A9)
* [عصارة خلوية](https://ar.wikipedia.org/wiki/%D8%B9%D8%B5%D8%A7%D8%B1%D8%A9_%D8%AE%D9%84%D9%88%D9%8A%D8%A9)
* [إجهاض](https://ar.wikipedia.org/wiki/%D8%A5%D8%AC%D9%87%D8%A7%D8%B6)
* [متدثرة تراخومية](https://ar.wikipedia.org/wiki/%D9%85%D8%AA%D8%AF%D8%AB%D8%B1%D8%A9_%D8%AA%D8%B1%D8%A7%D8%AE%D9%88%D9%85%D9%8A%D8%A9)
* [ورم منتبجات كلوي](https://ar.wikipedia.org/wiki/%D9%88%D8%B1%D9%85_%D9%85%D9%86%D8%AA%D8%A8%D8%AC%D8%A7%D8%AA_%D9%83%D9%84%D9%88%D9%8A)
* [علامة هوفر](https://ar.wikipedia.org/wiki/%D8%B9%D9%84%D8%A7%D9%85%D8%A9_%D9%87%D9%88%D9%81%D8%B1)
* [ورم الخلايا البدائية العصبية](https://ar.wikipedia.org/wiki/%D9%88%D8%B1%D9%85_%D8%A7%D9%84%D8%AE%D9%84%D8%A7%D9%8A%D8%A7_%D8%A7%D9%84%D8%A8%D8%AF%D8%A7%D8%A6%D9%8A%D8%A9_%D8%A7%D9%84%D8%B9%D8%B5%D8%A8%D9%8A%D8%A9)
* [إحليل تحتي](https://ar.wikipedia.org/wiki/%D8%A5%D8%AD%D9%84%D9%8A%D9%84_%D8%AA%D8%AD%D8%AA%D9%8A)
* [اقتباس من خارج السياق](https://ar.wikipedia.org/wiki/%D8%A7%D9%82%D8%AA%D8%A8%D8%A7%D8%B3_%D9%85%D9%86_%D8%AE%D8%A7%D8%B1%D8%AC_%D8%A7%D9%84%D8%B3%D9%8A%D8%A7%D9%82)
* [ستيوارت ناغل](https://ar.wikipedia.org/wiki/%D8%B3%D8%AA%D9%8A%D9%88%D8%A7%D8%B1%D8%AA_%D9%86%D8%A7%D8%BA%D9%84)


## astwiki: 147 
* [Odonata](https://ast.wikipedia.org/wiki/Odonata)
* [Anisoptera](https://ast.wikipedia.org/wiki/Anisoptera)
* [Marie Stopes](https://ast.wikipedia.org/wiki/Marie_Stopes)
* [Sistema inmunitariu](https://ast.wikipedia.org/wiki/Sistema_inmunitariu)
* [Ficus aurea](https://ast.wikipedia.org/wiki/Ficus_aurea)
* [Paullinia cupana](https://ast.wikipedia.org/wiki/Paullinia_cupana)
* [Llista de plantes melecinales (D-G)](https://ast.wikipedia.org/wiki/Llista_de_plantes_melecinales_%28D-G%29)
* [Rhododendron ponticum](https://ast.wikipedia.org/wiki/Rhododendron_ponticum)
* [Eicosanoide](https://ast.wikipedia.org/wiki/Eicosanoide)
* [Liliaceae](https://ast.wikipedia.org/wiki/Liliaceae)
* [Chelonia mydas](https://ast.wikipedia.org/wiki/Chelonia_mydas)
* [Trestornu llende de la personalidá](https://ast.wikipedia.org/wiki/Trestornu_llende_de_la_personalid%C3%A1)
* [Xakarta](https://ast.wikipedia.org/wiki/Xakarta)
* [Xeografía de los Países Baxos](https://ast.wikipedia.org/wiki/Xeograf%C3%ADa_de_los_Pa%C3%ADses_Baxos)
* [Turdus philomelos](https://ast.wikipedia.org/wiki/Turdus_philomelos)
* [Cervus elaphus](https://ast.wikipedia.org/wiki/Cervus_elaphus)
* [Apiales](https://ast.wikipedia.org/wiki/Apiales)
* [Aphloiaceae](https://ast.wikipedia.org/wiki/Aphloiaceae)
* [Arecaceae](https://ast.wikipedia.org/wiki/Arecaceae)
* [Nacionalismu lliberal](https://ast.wikipedia.org/wiki/Nacionalismu_lliberal)
* [Virus](https://ast.wikipedia.org/wiki/Virus)
* [Pseudociencia](https://ast.wikipedia.org/wiki/Pseudociencia)
* [Spindalis portoricensis](https://ast.wikipedia.org/wiki/Spindalis_portoricensis)
* [Orchidaceae](https://ast.wikipedia.org/wiki/Orchidaceae)
* [Gripe](https://ast.wikipedia.org/wiki/Gripe)
* [Antioxidante](https://ast.wikipedia.org/wiki/Antioxidante)
* [Otidiformes](https://ast.wikipedia.org/wiki/Otidiformes)
* [Biota del periodu Ediacáricu](https://ast.wikipedia.org/wiki/Biota_del_periodu_Ediac%C3%A1ricu)
* [Lanius excubitor](https://ast.wikipedia.org/wiki/Lanius_excubitor)
* [Phylloscopus bonelli](https://ast.wikipedia.org/wiki/Phylloscopus_bonelli)
* [Phylloscopus collybita](https://ast.wikipedia.org/wiki/Phylloscopus_collybita)
* [Phylloscopus orientalis](https://ast.wikipedia.org/wiki/Phylloscopus_orientalis)
* [Magnoliophyta](https://ast.wikipedia.org/wiki/Magnoliophyta)
* [Acoraceae](https://ast.wikipedia.org/wiki/Acoraceae)
* [Alismataceae](https://ast.wikipedia.org/wiki/Alismataceae)
* [Alstroemeriaceae](https://ast.wikipedia.org/wiki/Alstroemeriaceae)
* [Amaryllidaceae](https://ast.wikipedia.org/wiki/Amaryllidaceae)
* [Anarthriaceae](https://ast.wikipedia.org/wiki/Anarthriaceae)
* [Anthericaceae](https://ast.wikipedia.org/wiki/Anthericaceae)
* [Aponogetonaceae](https://ast.wikipedia.org/wiki/Aponogetonaceae)
* [Araceae](https://ast.wikipedia.org/wiki/Araceae)
* [Asteliaceae](https://ast.wikipedia.org/wiki/Asteliaceae)
* [Blandfordiaceae](https://ast.wikipedia.org/wiki/Blandfordiaceae)
* [Bromeliaceae](https://ast.wikipedia.org/wiki/Bromeliaceae)
* [Burmanniaceae](https://ast.wikipedia.org/wiki/Burmanniaceae)
* [Butomaceae](https://ast.wikipedia.org/wiki/Butomaceae)
* [Campynemataceae](https://ast.wikipedia.org/wiki/Campynemataceae)
* [Centrolepidaceae](https://ast.wikipedia.org/wiki/Centrolepidaceae)
* [Colchicaceae](https://ast.wikipedia.org/wiki/Colchicaceae)
* [Commelinaceae](https://ast.wikipedia.org/wiki/Commelinaceae)
* [Corsiaceae](https://ast.wikipedia.org/wiki/Corsiaceae)
* [Costaceae](https://ast.wikipedia.org/wiki/Costaceae)
* [Cyclanthaceae](https://ast.wikipedia.org/wiki/Cyclanthaceae)
* [Cymodoceaceae](https://ast.wikipedia.org/wiki/Cymodoceaceae)
* [Cyperaceae](https://ast.wikipedia.org/wiki/Cyperaceae)
* [Dasypogonaceae](https://ast.wikipedia.org/wiki/Dasypogonaceae)
* [Dioscoreaceae](https://ast.wikipedia.org/wiki/Dioscoreaceae)
* [Doryanthaceae](https://ast.wikipedia.org/wiki/Doryanthaceae)
* [Ecdeiocoleaceae](https://ast.wikipedia.org/wiki/Ecdeiocoleaceae)
* [Eriocaulaceae](https://ast.wikipedia.org/wiki/Eriocaulaceae)
* [Flagellariaceae](https://ast.wikipedia.org/wiki/Flagellariaceae)
* [Fumariaceae](https://ast.wikipedia.org/wiki/Fumariaceae)
* [Haemodoraceae](https://ast.wikipedia.org/wiki/Haemodoraceae)
* [Hanguanaceae](https://ast.wikipedia.org/wiki/Hanguanaceae)
* [Heliconiaceae](https://ast.wikipedia.org/wiki/Heliconiaceae)
* [Hydatellaceae](https://ast.wikipedia.org/wiki/Hydatellaceae)
* [Hydrocharitaceae](https://ast.wikipedia.org/wiki/Hydrocharitaceae)
* [Ixioliriaceae](https://ast.wikipedia.org/wiki/Ixioliriaceae)
* [Joinvilleaceae](https://ast.wikipedia.org/wiki/Joinvilleaceae)
* [Juncaceae](https://ast.wikipedia.org/wiki/Juncaceae)
* [Juncaginaceae](https://ast.wikipedia.org/wiki/Juncaginaceae)
* [Limnocharitaceae](https://ast.wikipedia.org/wiki/Limnocharitaceae)
* [Lowiaceae](https://ast.wikipedia.org/wiki/Lowiaceae)
* [Marantaceae](https://ast.wikipedia.org/wiki/Marantaceae)
* [Mayacaceae](https://ast.wikipedia.org/wiki/Mayacaceae)
* [Melanthiaceae](https://ast.wikipedia.org/wiki/Melanthiaceae)
* [Musaceae](https://ast.wikipedia.org/wiki/Musaceae)
* [Myrothamnaceae](https://ast.wikipedia.org/wiki/Myrothamnaceae)
* [Pandanaceae](https://ast.wikipedia.org/wiki/Pandanaceae)
* [Petermanniaceae](https://ast.wikipedia.org/wiki/Petermanniaceae)
* [Philesiaceae](https://ast.wikipedia.org/wiki/Philesiaceae)
* [Philydraceae](https://ast.wikipedia.org/wiki/Philydraceae)
* [Poaceae](https://ast.wikipedia.org/wiki/Poaceae)
* [Pontederiaceae](https://ast.wikipedia.org/wiki/Pontederiaceae)
* [Posidoniaceae](https://ast.wikipedia.org/wiki/Posidoniaceae)
* [Potamogetonaceae](https://ast.wikipedia.org/wiki/Potamogetonaceae)
* [Rapateaceae](https://ast.wikipedia.org/wiki/Rapateaceae)
* [Restionaceae](https://ast.wikipedia.org/wiki/Restionaceae)
* [Scheuchzeriaceae](https://ast.wikipedia.org/wiki/Scheuchzeriaceae)
* [Smilacaceae](https://ast.wikipedia.org/wiki/Smilacaceae)
* [Sparganiaceae](https://ast.wikipedia.org/wiki/Sparganiaceae)
* [Stemonaceae](https://ast.wikipedia.org/wiki/Stemonaceae)
* [Strelitziaceae](https://ast.wikipedia.org/wiki/Strelitziaceae)
* [Taccaceae](https://ast.wikipedia.org/wiki/Taccaceae)
* [Tecophilaeaceae](https://ast.wikipedia.org/wiki/Tecophilaeaceae)
* [Thismiaceae](https://ast.wikipedia.org/wiki/Thismiaceae)
* [Thurniaceae](https://ast.wikipedia.org/wiki/Thurniaceae)
* [Triuridaceae](https://ast.wikipedia.org/wiki/Triuridaceae)
* [Typhaceae](https://ast.wikipedia.org/wiki/Typhaceae)
* [Velloziaceae](https://ast.wikipedia.org/wiki/Velloziaceae)
* [Xyridaceae](https://ast.wikipedia.org/wiki/Xyridaceae)
* [Zannichelliaceae](https://ast.wikipedia.org/wiki/Zannichelliaceae)
* [Zingiberaceae](https://ast.wikipedia.org/wiki/Zingiberaceae)
* [Zosteraceae](https://ast.wikipedia.org/wiki/Zosteraceae)
* [Asparagales](https://ast.wikipedia.org/wiki/Asparagales)
* [Poales](https://ast.wikipedia.org/wiki/Poales)
* [Commelinidae](https://ast.wikipedia.org/wiki/Commelinidae)
* [Agavoideae](https://ast.wikipedia.org/wiki/Agavoideae)
* [Aphyllanthoideae](https://ast.wikipedia.org/wiki/Aphyllanthoideae)
* [Asphodeloideae](https://ast.wikipedia.org/wiki/Asphodeloideae)
* [Bataceae](https://ast.wikipedia.org/wiki/Bataceae)
* [Hamamelidae](https://ast.wikipedia.org/wiki/Hamamelidae)
* [Hemerocallidoideae](https://ast.wikipedia.org/wiki/Hemerocallidoideae)
* [Ranunculales](https://ast.wikipedia.org/wiki/Ranunculales)
* [Alismatales](https://ast.wikipedia.org/wiki/Alismatales)
* [Monocotyledoneae](https://ast.wikipedia.org/wiki/Monocotyledoneae)
* [Sistema de clasificación APG II](https://ast.wikipedia.org/wiki/Sistema_de_clasificaci%C3%B3n_APG_II)
* [Sistema de clasificación APG III](https://ast.wikipedia.org/wiki/Sistema_de_clasificaci%C3%B3n_APG_III)
* [Nefyn](https://ast.wikipedia.org/wiki/Nefyn)
* [Cetacea](https://ast.wikipedia.org/wiki/Cetacea)
* [Idioma mapuche](https://ast.wikipedia.org/wiki/Idioma_mapuche)
* [Idioma kawésqar](https://ast.wikipedia.org/wiki/Idioma_kaw%C3%A9sqar)
* [Filisteos](https://ast.wikipedia.org/wiki/Filisteos)
* [Perissodactyla](https://ast.wikipedia.org/wiki/Perissodactyla)
* [Panthera](https://ast.wikipedia.org/wiki/Panthera)
* [Non teísmu](https://ast.wikipedia.org/wiki/Non_te%C3%ADsmu)
* [Annie Lorrain Smith](https://ast.wikipedia.org/wiki/Annie_Lorrain_Smith)
* [V838 Monocerotis](https://ast.wikipedia.org/wiki/V838_Monocerotis)
* [Panthera pardus](https://ast.wikipedia.org/wiki/Panthera_pardus)
* [Carlemanniaceae](https://ast.wikipedia.org/wiki/Carlemanniaceae)
* [Sphenostemonaceae](https://ast.wikipedia.org/wiki/Sphenostemonaceae)
* [Rosales](https://ast.wikipedia.org/wiki/Rosales)
* [Sapindales](https://ast.wikipedia.org/wiki/Sapindales)
* [Flavonoide](https://ast.wikipedia.org/wiki/Flavonoide)
* [Puma concolor](https://ast.wikipedia.org/wiki/Puma_concolor)
* [Anisophylleaceae](https://ast.wikipedia.org/wiki/Anisophylleaceae)
* [Begoniaceae](https://ast.wikipedia.org/wiki/Begoniaceae)
* [Coriariaceae](https://ast.wikipedia.org/wiki/Coriariaceae)
* [Corynocarpaceae](https://ast.wikipedia.org/wiki/Corynocarpaceae)
* [Cucurbitaceae](https://ast.wikipedia.org/wiki/Cucurbitaceae)
* [Datiscaceae](https://ast.wikipedia.org/wiki/Datiscaceae)
* [Tetramelaceae](https://ast.wikipedia.org/wiki/Tetramelaceae)
* [Cucurbitales](https://ast.wikipedia.org/wiki/Cucurbitales)
* [Arnica montana](https://ast.wikipedia.org/wiki/Arnica_montana)
* [Sinocentrismu](https://ast.wikipedia.org/wiki/Sinocentrismu)
* [Melanina](https://ast.wikipedia.org/wiki/Melanina)
* [Ciclu sexual femenín](https://ast.wikipedia.org/wiki/Ciclu_sexual_femen%C3%ADn)


Skipping https://ban.wikipedia.org

...

## testwiki: 2 
* [User:Fhocutt (WMF)/Sandbox](https://test.wikipedia.org/wiki/User%3AFhocutt_%28WMF%29%2FSandbox)
* [Thesaurus](https://test.wikipedia.org/wiki/Thesaurus)


Skipping https://test2.wikipedia.org

Skipping https://test-commons.wikimedia.org

## Summary

|Wiki|Count|
|---|---|
|alswiki|5|
|arwiki|131|
|astwiki|147|
|barwiki|2|
|bgwiki|39|
|cawiki|137|
|cswiki|28|
|cywiki|1|
|dawiki|5|
|dewiki|248|
|dewikinews|5|
|dewikiversity|3|
|enwiki|1503|
|enwiktionary|4|
|enwikiquote|1|
|enwikivoyage|1|
|eowiki|64|
|eswiki|558|
|etwiki|11|
|fawiki|35|
|fiwiki|30|
|frwiki|238|
|fywiktionary|1|
|glwiki|125|
|hewiki|58|
|huwiki|62|
|hywiki|3|
|idwiki|1|
|itwiki|82|
|jawiki|100|
|kowiki|77|
|lgwiki|1|
|lvwiki|1|
|map_bmswiki|1|
|mkwiki|36|
|mlwiki|17|
|mnwiki|4|
|mswiki|45|
|nlwiki|74|
|nlwikibooks|1|
|nnwiki|21|
|nowiki|57|
|ocwiki|5|
|orwiki|3|
|pawiki|3|
|plwiki|82|
|plwikiquote|1|
|ptwiki|164|
|rowiki|22|
|ruwiki|157|
|shwiki|39|
|slwiki|29|
|sqwiki|2|
|srwiki|73|
|svwiki|93|
|tawiki|3|
|thwiki|33|
|trwiki|39|
|ukwiki|67|
|viwiki|105|
|zhwiki|142|
|commonswiki|3|
|metawiki|2|
|specieswiki|71|
|testwiki|2|

Total: 5103
```
{| class="wikitable"
|-
! Wiki !! Count !! Volunteer !! Progress
|-
| alswiki || 5 || || 
|-
| arwiki || 131 || || 
|-
| astwiki || 147 || || 
|-
| barwiki || 2 || || 
|-
| bgwiki || 39 || || 
|-
| cawiki || 137 || || 
|-
| cswiki || 28 || || 
|-
| cywiki || 1 || || 
|-
| dawiki || 5 || || 
|-
| dewiki || 248 || || 
|-
| dewikinews || 5 || || 
|-
| dewikiversity || 3 || || 
|-
| enwiki || 1503 || || 
|-
| enwiktionary || 4 || || 
|-
| enwikiquote || 1 || || 
|-
| enwikivoyage || 1 || || 
|-
| eowiki || 64 || || 
|-
| eswiki || 558 || || 
|-
| etwiki || 11 || || 
|-
| fawiki || 35 || || 
|-
| fiwiki || 30 || || 
|-
| frwiki || 238 || || 
|-
| fywiktionary || 1 || || 
|-
| glwiki || 125 || || 
|-
| hewiki || 58 || || 
|-
| huwiki || 62 || || 
|-
| hywiki || 3 || || 
|-
| idwiki || 1 || || 
|-
| itwiki || 82 || || 
|-
| jawiki || 100 || || 
|-
| kowiki || 77 || || 
|-
| lgwiki || 1 || || 
|-
| lvwiki || 1 || || 
|-
| map_bmswiki || 1 || || 
|-
| mkwiki || 36 || || 
|-
| mlwiki || 17 || || 
|-
| mnwiki || 4 || || 
|-
| mswiki || 45 || || 
|-
| nlwiki || 74 || || 
|-
| nlwikibooks || 1 || || 
|-
| nnwiki || 21 || || 
|-
| nowiki || 57 || || 
|-
| ocwiki || 5 || || 
|-
| orwiki || 3 || || 
|-
| pawiki || 3 || || 
|-
| plwiki || 82 || || 
|-
| plwikiquote || 1 || || 
|-
| ptwiki || 164 || || 
|-
| rowiki || 22 || || 
|-
| ruwiki || 157 || || 
|-
| shwiki || 39 || || 
|-
| slwiki || 29 || || 
|-
| sqwiki || 2 || || 
|-
| srwiki || 73 || || 
|-
| svwiki || 93 || || 
|-
| tawiki || 3 || || 
|-
| thwiki || 33 || || 
|-
| trwiki || 39 || || 
|-
| ukwiki || 67 || || 
|-
| viwiki || 105 || || 
|-
| zhwiki || 142 || || 
|-
| commonswiki || 3 || || 
|-
| metawiki || 2 || || 
|-
| specieswiki || 71 || || 
|-
| testwiki || 2 || || 
|}
```


Finished

### Licensing
Copyright 2019 AntiCompositeNumber

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

  http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

### To fork:
1. Open the [raw page](?format=raw) and save it to your computer
2. Go to your [PAWS control panel](https://paws.wmflabs.org/paws/hub) and sign in using OAUTH
3. Click Upload and upload the file from step 1
4. To run, open the notebook and click Cell > Run All